Mit diesem Jupyter Notbook werden bibl. Daten der Manifestationsebene von der SRU-K10plus-Schnittstelle http://sru.k10plus.de/opac-de-627 abgerufen und Datenfelder der SRU-Antwort mit Hilfe von XPath-Abfragen gezielt durchsucht sowie deren Inhalt an eine CSV-Datei übergeben. Das Notebook ist im Rahmen der Masterarbeit "Digitale Erschließung, Analyse und Visualisierung der
Handbibliothek von Herzogin Philippine Charlotte von Braunschweig-Lüneburg. Forschungsdatenmanagement in Kooperation
zwischen Forschung und Bibliothek." im Studiengang Digitales Datenmanagement der FH Potsdam/HU Berlin entstanden.

Die nachfolgenden Funktionen sind zum weiteren Verständnis auskommentiert.
Ich danke Dario Kampkasper (DK) für seine Expertise und Unterstützung beim Erstellen des Codes.
20.02.2023, Henrike Fricke-Steyer


In [ ]:
#Import der benötigten Python-Bibliothken
import pandas as pd
import requests 
from lxml import etree
import csv

In [81]:
#Einlesen einer Datei mit Identifikationsnummern (IDNs) und Umwandlung der Spalte IDN in Liste die auf dem eigenen 
#Computer liegt, der Pfad "C:/...csv" muss entsprechend ersetzt werden: 

colnames=['IDN']

ns = {
    "pica": "info:srw/schema/5/picaXML-v1.0"
}

#Einlesen einer Datei mit Identifikationsnummern (IDNs) und Umwandlung der Spalte IDN in Liste die auf dem eigenen 
#Desktop liegt: 
data = pd.read_csv("C:/...csv", names=colnames, header=None)
idns = data['IDN'].to_list()
print(len(idns))

1644


In [82]:
#gekürzte Funktion zur Abfrage der IDNs aus dem SRU-Tutorial (www.dnb.de/dnblabtutorials)
#Schleife zur SRU-Abfrage, spricht die Schnittstelle an, Parameter in entsprechender Reihenfolge angepasst
def sru(query):
    
    base_url = "http://sru.k10plus.de/opac-de-627"
    params = {'version': '1.1',
              'operation': 'searchRetrieve',                    
              'query': query,
              'maximumRecords': '100',
              'recordSchema' : 'picaxml'
         }
    r = requests.get(base_url, params=params)
   
    ns = {
        "zs":"http://www.loc.gov/zing/srw/",
        "pica": "info:srw/schema/5/picaXML-v1.0"
    }
    xml = etree.fromstring(r.content)
    records = xml.xpath("//zs:recordData/pica:record", namespaces=ns)
    
    return records
    

In [83]:
#Übergabe der einzelnen Elemente aus der IDN-Liste an die Funktion, um für jedes Element eine SRU-Abfrage 
#auszulösen. Die Ergebnisse der einzelnen Abfragen werden als Liste in der Variable "response" gespeichert
response = [sru(searchtext) for searchtext in idns]

#Ausgabe der Anzahl der in der Liste enthaltenen Elemente - diese sollte mit der Anzahl der abgefragten
#IDNs übereinstimmen. 
print (len(response))

1644


In [84]:
#Ausgabe der gesammelten Treffer in einer Datei:
# 2022-10-21 DK
resultTree = etree.ElementTree()
recordsElement = etree.Element('records')
resultTree._setroot(recordsElement)

for record in response: 
    recordsElement.append(record[0])

with open ('data.xml', 'wb',) as f:
    resultTree.write(f, encoding='utf-8')

In [85]:
# Hilfsfunktion, um ein wiederholbares Fedl abzurufen (inkl. Unterfelder)
# 2023-01-21 DK
def get_repeatable ( xml, *fields ):
    values = []
    for field in fields:
        xpath = "pica:datafield[@tag='" + field + "']"
        values.extend(xml.xpath(xpath, namespaces=ns))
    
    return values

In [86]:
# Hilfsfunktion, um für eine Liste von Unterfeldern die Werte abzurufen
# 2023-01-21 DK
def get_subfields ( xml, subfields ):
    values = []
    for subfield in subfields:
        xpath = "pica:subfield[@code = '" + subfield + "']/text()"
        try:
            val = xml.xpath(xpath, namespaces=ns)
            values.append(val[0])
        except:
            "leer"
    
    if len(values):
        return values[0]
    else:
        return "leer"

In [87]:
# Hilfsfunktion, um Unterfelder gezielt abzurufen
# 2023-01-21 DK
# Es wird immer nur das erste gefundene zurückgegeben, daher nur für nicht wiederholbare Felder verwenden
def get_field ( xml, field, subfields ):
    values = []
    for subfield in subfields:
        xpath = "pica:datafield[@tag='" + field + "']/pica:subfield[@code = '" + subfield + "']/text()"
        try:
            value = xml.xpath(xpath, namespaces=ns)[0]
            values.append(value)
        except:
            "leer"
    
    try:
        return next(value for value in values if value)
    except:
        "leer"

In [91]:
# Extraktion entsprechender Inhalte aus den einzelnen Records: 
# Funktion 2022-10-21 DK

def parse_record(record):
    xml = record[0]


    # bibl. Gattung und Status / 500 
    bgs = xml.xpath("pica:datafield[@tag='002@']/pica:subfield[@code = '0']", namespaces=ns)
    try:
        bgs = bgs[0].text
    except:
        bgs = ''
        
    # Erscheinungsjahr / 1100
    erj = xml.xpath("pica:datafield[@tag='011@']/pica:subfield[@code = 'a']", namespaces=ns)
    try:
        erj = erj[0].text
    except:
        erj = ''
        
    # Erscheinungsort / 4030
    eo = xml.xpath("pica:datafield[@tag='033A']/pica:subfield[@code = 'p']", namespaces=ns)
    try:
        eo = eo[0].text
    except:
        eo = ''
        
    # Erscheinungsort GND-Nummer / 4030
    eog = xml.xpath("pica:datafield[@tag='033D']/pica:subfield[@code = '7']", namespaces=ns)
    try:
        eog = eog[0].text
    except:
        eog = ''
    
   # Sprachcode / 1500
    spr = xml.xpath("pica:datafield[@tag='010@']/pica:subfield[@code = 'a']", namespaces=ns)
    try:
        spr = spr[0].text
    except:
        spr = ''
    
    # ppn / 100
    idn = xml.xpath("pica:datafield[@tag='003@']/pica:subfield[@code = '0']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = ''
    
    # 2023-01-21 DK:
   # → in titelKategorien eine Liste mit den Feldern erstellen und nach Gewichtung sortieren
    titelKategorien = [ '021A', '027A', '046C', '036E', '036C' ] # 027A=3260, 046C=4212, 036E=4170–4179, 036C=4150
    titelListe = []
    for titelKategorie in titelKategorien:
        # relevante Unterfelder abrufen
        titelFelder = [
            get_field(xml, titelKategorie, 'a'),
            get_field(xml, titelKategorie, 'd'),
            get_field(xml, titelKategorie, 'f')
        ]
        titelei = [ti.replace('@', '') for ti in titelFelder if ti]
        # vorhandene Unterfelder mit ' ; ' zusammensetzen
        titelListe.append(" ; ".join(titelei))
    # 2023-01-21 DK Ende

    try:
        titel = next(sub for sub in titelListe if sub)
    except:
        titel = ""
    
    # 2021-01-21 DK
    
    # 028A=3000 wird nicht wiederholt; daher nur eine Abfrage
    verfassername = get_field(xml, '028A', ['a', 'A', 'l'])
    verfasservorname = get_field(xml, '028A', ['d', 'D', 'P'])
    gndverfasser = get_field(xml, '028A', '7')

# Weitere Verfasser 028B=3001,3002 können wiederholt werden
    weitereVerfKategorien = get_repeatable(xml, '028B')
    weitereVerf = []
    for kat in weitereVerfKategorien:
        weitereVerf.append([
            get_subfields(kat, ['a', 'A', 'l']),
            get_subfields(kat, ['d', 'D', 'P']),
            get_subfields(kat, ['7'])
        ])
    
    vf1n = ""
    vf1v = ""
    vf1g = ""
    vf2n = ""
    vf2v = ""
    vf2g = ""

    try:
        vf1n = weitereVerf[0][0]
        vf1v = weitereVerf[0][1]
        vf1g = weitereVerf[0][2]
        vf2n = weitereVerf[1][0]
        vf2v = weitereVerf[1][1]
        vf2g = weitereVerf[1][2]
    except:
        pass
    
# Sonstige beteiligte Personen 028C=3010, können wiederholt werden
    weitereVerfpKategorien = get_repeatable(xml, '028C')
    weitereVerfp = []
    for kat in weitereVerfpKategorien:
        weitereVerfp.append([
            get_subfields(kat, ['a', 'A', 'l']),
            get_subfields(kat, ['d', 'D', 'P']),
            get_subfields(kat, ['7'])
        ])
    
    pvf1n = ""
    pvf1v = ""
    pvf1g = ""
    pvf2n = ""
    pvf2v = ""
    pvf2g = ""

    try:
        pvf1n = weitereVerfp[0][0]
        pvf1v = weitereVerfp[0][1]
        pvf1g = weitereVerfp[0][2]
        pvf2n = weitereVerfp[1][0]
        pvf2v = weitereVerfp[1][1]
        pvf2g = weitereVerfp[1][2]
    except:
        pass 

 # Sonstige beteiligte Personen, 028G=3050 können wiederholt werden
    weiterePersKategorien = get_repeatable(xml, '028G')
    weiterePers = []
    for kat in weiterePersKategorien:
        weiterePers.append([
            get_subfields(kat, ['a', 'A', 'l']),
            get_subfields(kat, ['d', 'D', 'P']),
            get_subfields(kat, ['7'])
        ])
    
    pers1n = ""
    pers1v = ""
    pers1g = ""
    pers2n = ""
    pers2v = ""
    pers2g = ""
     

    try:
        pers1n = weiterePers[0][0]
        pers1v = weiterePers[0][1]
        pers1g = weiterePers[0][2]
        pers2n = weiterePers[1][0]
        pers2v = weiterePers[1][1]
        pers2g = weiterePers[1][2]
    except:
        pass
    
       
              
    
    meta_dict = {
        "PPN":idn,
        "Bibliographische Gattung und Status":bgs,
        "Sprache":spr,
        "Erscheinungsjahr":erj,
        "Erscheinungsort":eo,
        "Erscheinungsort GND":eog,
        "Titel":titel,
        "Verfasservorname":verfasservorname,
        "Verfassername":verfassername,
        "GND Verfasser":gndverfasser,
        "1. Weitere Verfasser Vorname":vf1v,
        "1. Weitere Verfasser Nachname":vf1n,
        "1. Weitere Verfasser GND":vf1g,
        "2. Weitere Verfasser Vorname":vf2v,
        "2. Weitere Verfasser Nachname":vf2n,
        "2. Weitere Verfasser GND":vf2g,
        
        "1. Weitere Person Vorname":pvf1v,
        "1. Weitere Person Nachname":pvf1n,
        "1. Weitere Person GND":pvf1g,
        "2. Weitere Person Vorname":pvf2v,
        "2. Weitere Person Nachname":pvf2n,
        "2. Weitere Person GND":pvf2g,
        
        "1. Sonstige Person Vorname":pers1v,
        "1. Sonstige Person Nachname":pers1n,
        "1. Sonstige Person GND":pers1g,
        "2. Sonstige Person Vorname":pers2v,
        "2. Sonstige Person Nachname":pers2n,
        "2. Sonstige Person GND":pers2g
       
    }
    
    return meta_dict

In [92]:
#Übergabe der einzelnen Records an die Funktion und Ausgabe als DataFrame:
output = [parse_record(record) for record in response]

# print(output[0])
df = pd.DataFrame(output)

In [93]:
#Ausgabe des DataFrames in CSV-Datei
df.to_csv('sru_antwort.csv', sep ='\t') 